In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
stops = pd.read_csv("stops.txt")
stops.head()

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,zone_id,stop_IBNR,stop_PKPPLK,platform_code,wheelchair_boarding
0,100101,Kijowska 01,52.248455,21.044827,NaN,NaN,1,NaN,NaN,NaN,1
1,100102,Kijowska 02,52.249078,21.044443,NaN,NaN,1,NaN,NaN,NaN,1
2,100103,Kijowska 03,52.248928,21.044169,NaN,NaN,1,NaN,NaN,NaN,1
3,100104,Kijowska 04,52.249969,21.041588,NaN,NaN,1,NaN,NaN,NaN,1
4,100106,Kijowska 06,52.250078,21.043848,NaN,NaN,1,NaN,NaN,NaN,1


In [3]:
stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   stop_id              6992 non-null   object 
 1   stop_name            6992 non-null   object 
 2   stop_lat             6992 non-null   float64
 3   stop_lon             6992 non-null   float64
 4   location_type        365 non-null    float64
 5   parent_station       308 non-null    object 
 6   zone_id              6992 non-null   object 
 7   stop_IBNR            101 non-null    float64
 8   stop_PKPPLK          101 non-null    float64
 9   platform_code        2 non-null      object 
 10  wheelchair_boarding  6992 non-null   int64  
dtypes: float64(5), int64(1), object(5)
memory usage: 601.0+ KB


In [4]:
stops = stops.drop(columns = ['location_type', 'stop_IBNR', 'stop_PKPPLK', 'platform_code', 'wheelchair_boarding'])

In [5]:
stops.describe()

,stop_lat,stop_lon
count,6992.000000,6992.000000
mean,52.227577,21.021932
std,0.095568,0.123988
min,51.938106,20.462591
25%,52.178033,20.948971
50%,52.233202,21.016604
75%,52.280610,21.089016
max,52.483010,21.463783


In [6]:
stops['stop_id'].nunique()

6992

In [7]:
stops_unique_ids = stops['stop_id'].unique()

In [8]:
stops_time = pd.read_csv("stop_times.txt")

C:\Users\97252\AppData\Local\Temp\ipykernel_20980\2039050293.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  stops_time = pd.read_csv("stop_times.txt")


In [9]:
stops_time.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,RA210929/1/TD-3ANNO/DP/04.13,04:16:00,04:16:00,324004,0,0.0,1.0,0.0000
1,RA210929/1/TD-3ANNO/DP/04.13,04:18:00,04:18:00,323904,1,0.0,0.0,0.5696
2,RA210929/1/TD-3ANNO/DP/04.13,04:20:00,04:20:00,323804,2,0.0,0.0,0.9601
3,RA210929/1/TD-3ANNO/DP/04.13,04:21:00,04:21:00,323704,3,0.0,0.0,1.3816
4,RA210929/1/TD-3ANNO/DP/04.13,04:22:00,04:22:00,328604,4,0.0,0.0,1.7440


In [10]:
stops_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5037616 entries, 0 to 5037615
Data columns (total 8 columns):
 #   Column               Dtype  
---  ------               -----  
 0   trip_id              object 
 1   arrival_time         object 
 2   departure_time       object 
 3   stop_id              object 
 4   stop_sequence        int64  
 5   pickup_type          float64
 6   drop_off_type        float64
 7   shape_dist_traveled  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 307.5+ MB


In [11]:
stops_time['stop_id'].nunique()

10427

In [12]:
stops_time['stop_id'].unique()

array([324004, 323904, 323804, ..., '1526mP2', '1137mP2', '1140mP2'],
      dtype=object)

In [13]:
stops_time_ids = stops_time['stop_id'].unique()

In [14]:
# Function for cleaning stop ids:
def clean_stop_id(stop):
    match = re.search(r'\d+', str(stop))  
    return match.group(0) if match else None  

# stops.txt
stops['clean_stop_id'] = stops['stop_id'].apply(clean_stop_id)
stops_cleaned = set(stops['clean_stop_id'].dropna())

In [15]:
stops['stop_id'].nunique()

6992

In [16]:
stops_time['stop_id'] = stops_time['stop_id'].astype(str)
stop_times_set = set(stops_time['stop_id'])

In [17]:
# Removing all letters and symbols:
def remove_letters(stop_id):
    return re.sub(r'\D', '', stop_id)  


stops["stop_id"] = stops["stop_id"].astype(str).apply(remove_letters)
stops_time["stop_id"] = stops_time["stop_id"].astype(str).apply(remove_letters)

In [18]:
stops['stop_id_numeric'] = stops['stop_id'].str.extract(r'(\d+)')  
stops['stop_id_numeric'] = stops['stop_id_numeric'].astype(str)  

In [19]:
# Сalculating coincidences:
matching_ids = set(stops["stop_id"]) & set(stops_time["stop_id"])
missing_in_stops = set(stops_time["stop_id"]) - set(stops["stop_id"])
missing_in_stop_times = set(stops["stop_id"]) - set(stops_time["stop_id"])

In [20]:
print(f" Number of concidenced stop_id: {len(matching_ids)}")
print(f" Number stop_id, that are not in stops.txt: {len(missing_in_stops)}")
print(f" Number stop_id, that are not in stop_times.txt: {len(missing_in_stop_times)}")

 Number of concidenced stop_id: 6706
 Number stop_id, that are not in stops.txt: 0
 Number stop_id, that are not in stop_times.txt: 250


Checking hypotesis that these stops belongs to metro

In [21]:
stops_time["hour"] = stops_time["arrival_time"].str[:2].astype(int)
stops_time.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,hour
0,RA210929/1/TD-3ANNO/DP/04.13,04:16:00,04:16:00,324004,0,0.0,1.0,0.0000,4
1,RA210929/1/TD-3ANNO/DP/04.13,04:18:00,04:18:00,323904,1,0.0,0.0,0.5696,4
2,RA210929/1/TD-3ANNO/DP/04.13,04:20:00,04:20:00,323804,2,0.0,0.0,0.9601,4
3,RA210929/1/TD-3ANNO/DP/04.13,04:21:00,04:21:00,323704,3,0.0,0.0,1.3816,4
4,RA210929/1/TD-3ANNO/DP/04.13,04:22:00,04:22:00,328604,4,0.0,0.0,1.7440,4


In [22]:
trips = pd.read_csv("trips.txt")
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id,exceptional,wheelchair_accessible,bikes_allowed
0,1,RA210929/1/DP,RA210929/1/TD-3ANNO/DP/04.13,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
1,1,RA210929/1/DP,RA210929/1/TD-3ANNO/DP/04.32,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
2,1,RA210929/1/DP,RA210929/1/TD-3ANNO/DP/05.12,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
3,1,RA210929/1/DP,RA210929/1/TD-3ANNO/DP/05.43,Annopol,0,RA210929/1/TD-3ANNO,1,2,1
4,1,RA210929/1/DP,RA210929/1/TD-3ANNO/DP/06.08,Annopol,0,RA210929/1/TD-3ANNO,1,1,1


In [23]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180186 entries, 0 to 180185
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   route_id               180186 non-null  object
 1   service_id             180186 non-null  object
 2   trip_id                180186 non-null  object
 3   trip_headsign          180186 non-null  object
 4   direction_id           180186 non-null  int64 
 5   shape_id               180186 non-null  object
 6   exceptional            180186 non-null  int64 
 7   wheelchair_accessible  180186 non-null  int64 
 8   bikes_allowed          180186 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 12.4+ MB


In [24]:
stops_time['trip_id'] = stops_time['trip_id'].str.strip().str.replace(r'_$', '', regex=True)
trips['trip_id'] = trips['trip_id'].str.strip().str.replace(r'_$', '', regex=True)

In [25]:
stop_trips_with_routes = stops_time.merge(trips, how='left', on='trip_id')

In [26]:
stop_trips_with_routes.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled,hour,route_id,service_id,trip_headsign,direction_id,shape_id,exceptional,wheelchair_accessible,bikes_allowed
0,RA210929/1/TD-3ANNO/DP/04.13,04:16:00,04:16:00,324004,0,0.0,1.0,0.0000,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
1,RA210929/1/TD-3ANNO/DP/04.13,04:18:00,04:18:00,323904,1,0.0,0.0,0.5696,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
2,RA210929/1/TD-3ANNO/DP/04.13,04:20:00,04:20:00,323804,2,0.0,0.0,0.9601,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
3,RA210929/1/TD-3ANNO/DP/04.13,04:21:00,04:21:00,323704,3,0.0,0.0,1.3816,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,1,1
4,RA210929/1/TD-3ANNO/DP/04.13,04:22:00,04:22:00,328604,4,0.0,0.0,1.7440,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,1,1


In [27]:
routes = pd.read_csv('routes.txt')
routes.head()

,agency_id,route_id,route_short_name,route_long_name,route_type,route_color,route_text_color,route_sort_order
0,0,1,1,Banacha — Annopol,0,B60000,FFFFFF,2
1,0,2,2,Winnica — Metro Młociny,0,B60000,FFFFFF,3
2,0,3,3,Gocławek — Annopol,0,B60000,FFFFFF,4
3,0,4,4,Żerań Wschodni — Wyścigi,0,B60000,FFFFFF,5
4,0,6,6,Metro Młociny — Gocławek,0,B60000,FFFFFF,6


In [28]:
routes['route_type'].unique()

array([0, 3, 2, 1], dtype=int64)

In [29]:
routes = routes.drop(columns=['agency_id', 'route_color', 'route_text_color', 'route_sort_order'])

In [30]:
stops_with_types = stop_trips_with_routes.merge(routes, how='left', on='route_id')

In [31]:
metro = stops_with_types[stops_with_types['route_type']==1]
metro['stop_id'].nunique()

34

In [32]:
stops_with_types = stops_with_types.drop(columns=['shape_dist_traveled', 'wheelchair_accessible', 'bikes_allowed', 'route_short_name'])

In [33]:
stops_total_info = stops_with_types.merge(stops, how='left', on='stop_id')

In [34]:
stops_total_info = stops_with_types.merge(stops, left_on='stop_id', right_on='stop_id_numeric', how='left')


In [35]:
stops_total_info.count()

trip_id            5042371
arrival_time       5042371
departure_time     5042371
stop_id_x          5042371
stop_sequence      5042371
pickup_type        5041827
drop_off_type      5041827
hour               5042371
route_id           5042371
service_id         5042371
trip_headsign      5042371
direction_id       5042371
shape_id           5042371
exceptional        5042371
route_long_name    5042371
route_type         5042371
stop_id_y          5042371
stop_name          5042371
stop_lat           5042371
stop_lon           5042371
parent_station       17509
zone_id            5042371
clean_stop_id      5042371
stop_id_numeric    5042371
dtype: int64

In [36]:
frequencies = pd.read_csv("frequencies.txt", dtype={'trip_id': str})
frequencies.head()

,trip_id,start_time,end_time,headway_secs,exact_times
0,M1/TP-MLO/M1,05:00:00,05:32:59,360,0
1,M1/TP-MLO/M1,05:33:00,05:38:59,210,0
2,M1/TP-MLO/M1,05:39:00,08:49:59,150,0
3,M1/TP-MLO/M1,08:50:00,14:08:59,210,0
4,M1/TP-MLO/M1,14:09:00,19:11:59,150,0


In [37]:
frequencies = frequencies.merge(trips[['trip_id', 'route_id']], on='trip_id', how='left')

In [38]:
frequencies = frequencies.merge(routes[['route_id', 'route_type']], on='route_id', how='left')

In [39]:
frequencies.head()

,trip_id,start_time,end_time,headway_secs,exact_times,route_id,route_type
0,M1/TP-MLO/M1,05:00:00,05:32:59,360,0,M1,1
1,M1/TP-MLO/M1,05:33:00,05:38:59,210,0,M1,1
2,M1/TP-MLO/M1,05:39:00,08:49:59,150,0,M1,1
3,M1/TP-MLO/M1,08:50:00,14:08:59,210,0,M1,1
4,M1/TP-MLO/M1,14:09:00,19:11:59,150,0,M1,1


In [40]:
frequencies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101 entries, 0 to 100
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   trip_id       101 non-null    object
 1   start_time    101 non-null    object
 2   end_time      101 non-null    object
 3   headway_secs  101 non-null    int64 
 4   exact_times   101 non-null    int64 
 5   route_id      101 non-null    object
 6   route_type    101 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 6.3+ KB


In [41]:
print(frequencies.columns)
print(routes.columns)


Index(['trip_id', 'start_time', 'end_time', 'headway_secs', 'exact_times',
       'route_id', 'route_type'],
      dtype='object')
Index(['route_id', 'route_short_name', 'route_long_name', 'route_type'], dtype='object')


In [42]:
# Function for generation metro trips
def generate_metro_trips(row):
    times = pd.date_range(start=row['start_time'], end=row['end_time'], freq=f"{row['headway_secs']}S").time
    return pd.DataFrame({'trip_id': row['trip_id'], 'departure_time': times, 'route_id': row['route_id'], 'route_type': row['route_type']})

In [43]:
def fix_large_hours(time_str):
    """
    Fixing time errors (for example, 25:30:00 → 01:30:00).
    """
    if isinstance(time_str, str):
        match = re.match(r'(\d+):(\d{2}):(\d{2})', time_str)
        if match:
            hour, minute, second = map(int, match.groups())
            if hour >= 24:
                hour -= 24  
                return f"{hour:02}:{minute:02}:{second:02}"
    return time_str  

frequencies["start_time"] = frequencies["start_time"].apply(fix_large_hours)
frequencies["end_time"] = frequencies["end_time"].apply(fix_large_hours)

print("Times errors are fixed")

Times errors are fixed


In [44]:
metro_trips = pd.concat(frequencies.apply(generate_metro_trips, axis=1).tolist(), ignore_index=True)
print("Metro trips are generated")
print(metro_trips.head())

Metro trips are generated
        trip_id departure_time route_id  route_type
0  M1/TP-MLO/M1       05:00:00       M1           1
1  M1/TP-MLO/M1       05:06:00       M1           1
2  M1/TP-MLO/M1       05:12:00       M1           1
3  M1/TP-MLO/M1       05:18:00       M1           1
4  M1/TP-MLO/M1       05:24:00       M1           1


In [45]:
metro_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   trip_id         3810 non-null   object
 1   departure_time  3810 non-null   object
 2   route_id        3810 non-null   object
 3   route_type      3810 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 119.2+ KB


In [46]:
metro_trips["departure_time"] = metro_trips["departure_time"].astype(str)
metro_trips["hour"] = metro_trips["departure_time"].str[:2].astype(int) 

In [47]:
metro_trips["hour"] = metro_trips["departure_time"].str[:2].astype(int) 

In [48]:
metro_trips.head()

,trip_id,departure_time,route_id,route_type,hour
0,M1/TP-MLO/M1,05:00:00,M1,1,5
1,M1/TP-MLO/M1,05:06:00,M1,1,5
2,M1/TP-MLO/M1,05:12:00,M1,1,5
3,M1/TP-MLO/M1,05:18:00,M1,1,5
4,M1/TP-MLO/M1,05:24:00,M1,1,5


In [49]:
metro_hourly = metro_trips.groupby(["route_id", "hour"]).size().reset_index(name="number_of_trips")

In [50]:
metro_hourly.head()

,route_id,hour,number_of_trips
0,M1,0,14
1,M1,1,16
2,M1,2,4
3,M1,5,101
4,M1,6,133


In [51]:
metro_data = stops_total_info[stops_total_info['route_type']==1].copy()

In [52]:
metro_data.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id_x',
       'stop_sequence', 'pickup_type', 'drop_off_type', 'hour', 'route_id',
       'service_id', 'trip_headsign', 'direction_id', 'shape_id',
       'exceptional', 'route_long_name', 'route_type', 'stop_id_y',
       'stop_name', 'stop_lat', 'stop_lon', 'parent_station', 'zone_id',
       'clean_stop_id', 'stop_id_numeric'],
      dtype='object')

In [53]:
metro_data

,trip_id,arrival_time,departure_time,stop_id_x,stop_sequence,pickup_type,drop_off_type,hour,route_id,service_id,...,route_long_name,route_type,stop_id_y,stop_name,stop_lat,stop_lon,parent_station,zone_id,clean_stop_id,stop_id_numeric
5041827,M1/TP-MLO/M1,00:00:00,00:00:00,32821,0,NaN,NaN,0,M1,M1,...,Kabaty — Młociny,1,32821,Kabaty,52.131983,21.065174,3282m,1,3282,32821
5041828,M1/TP-MLO/M1,00:00:00,00:00:00,32821,0,NaN,NaN,0,M1,M1,...,Kabaty — Młociny,1,32821,1,52.131621,21.065989,3282m,1,3282,32821
5041829,M1/TP-MLO/M1,00:02:00,00:02:00,32811,1,NaN,NaN,0,M1,M1,...,Kabaty — Młociny,1,32811,Natolin,52.141008,21.056510,3281m,1,3281,32811
5041830,M1/TP-MLO/M1,00:02:00,00:02:00,32811,1,NaN,NaN,0,M1,M1,...,Kabaty — Młociny,1,32811,1,52.140646,21.057267,3281m,1,3281,32811
5041831,M1/TP-MLO/M1,00:04:00,00:04:00,32801,2,NaN,NaN,0,M1,M1,...,Kabaty — Młociny,1,32801,Imielin,52.149136,21.046366,3280m,1,3280,32801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5042366,M2/TP-KSJ/M7,00:19:00,00:19:00,50052,10,NaN,NaN,0,M2,M7,...,Księcia Janusza — Trocka,1,50052,2,52.231686,20.966844,5005m,1,5005,50052
5042367,M2/TP-KSJ/M7,00:21:00,00:21:00,50282,11,NaN,NaN,0,M2,M7,...,Księcia Janusza — Trocka,1,50282,Młynów,52.237640,20.960475,5028m,1,5028,50282
5042368,M2/TP-KSJ/M7,00:21:00,00:21:00,50282,11,NaN,NaN,0,M2,M7,...,Księcia Janusza — Trocka,1,50282,2,52.237671,20.961693,5028m,1,5028,50282
5042369,M2/TP-KSJ/M7,00:23:00,00:23:00,50302,12,NaN,NaN,0,M2,M7,...,Księcia Janusza — Trocka,1,50302,Księcia Janusza,52.239096,20.944110,5030m,1,5030,50302


In [54]:
metro_data = metro_data.drop(columns=['hour'])

In [55]:
metro_data["hour"] = stops_total_info["departure_time"].str.split(":").str[1].astype(int)
metro_data

,trip_id,arrival_time,departure_time,stop_id_x,stop_sequence,pickup_type,drop_off_type,route_id,service_id,trip_headsign,...,route_type,stop_id_y,stop_name,stop_lat,stop_lon,parent_station,zone_id,clean_stop_id,stop_id_numeric,hour
5041827,M1/TP-MLO/M1,00:00:00,00:00:00,32821,0,NaN,NaN,M1,M1,Młociny,...,1,32821,Kabaty,52.131983,21.065174,3282m,1,3282,32821,0
5041828,M1/TP-MLO/M1,00:00:00,00:00:00,32821,0,NaN,NaN,M1,M1,Młociny,...,1,32821,1,52.131621,21.065989,3282m,1,3282,32821,0
5041829,M1/TP-MLO/M1,00:02:00,00:02:00,32811,1,NaN,NaN,M1,M1,Młociny,...,1,32811,Natolin,52.141008,21.056510,3281m,1,3281,32811,2
5041830,M1/TP-MLO/M1,00:02:00,00:02:00,32811,1,NaN,NaN,M1,M1,Młociny,...,1,32811,1,52.140646,21.057267,3281m,1,3281,32811,2
5041831,M1/TP-MLO/M1,00:04:00,00:04:00,32801,2,NaN,NaN,M1,M1,Młociny,...,1,32801,Imielin,52.149136,21.046366,3280m,1,3280,32801,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5042366,M2/TP-KSJ/M7,00:19:00,00:19:00,50052,10,NaN,NaN,M2,M7,Księcia Janusza,...,1,50052,2,52.231686,20.966844,5005m,1,5005,50052,19
5042367,M2/TP-KSJ/M7,00:21:00,00:21:00,50282,11,NaN,NaN,M2,M7,Księcia Janusza,...,1,50282,Młynów,52.237640,20.960475,5028m,1,5028,50282,21
5042368,M2/TP-KSJ/M7,00:21:00,00:21:00,50282,11,NaN,NaN,M2,M7,Księcia Janusza,...,1,50282,2,52.237671,20.961693,5028m,1,5028,50282,21
5042369,M2/TP-KSJ/M7,00:23:00,00:23:00,50302,12,NaN,NaN,M2,M7,Księcia Janusza,...,1,50302,Księcia Janusza,52.239096,20.944110,5030m,1,5030,50302,23


In [56]:
metro_data = metro_data.drop(columns=['trip_id', 'arrival_time', 'departure_time', 'stop_id_x',
                                      'pickup_type', 'drop_off_type', 'stop_id_y', 'clean_stop_id'])

In [57]:
metro_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544 entries, 5041827 to 5042370
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   stop_sequence    544 non-null    int64  
 1   route_id         544 non-null    object 
 2   service_id       544 non-null    object 
 3   trip_headsign    544 non-null    object 
 4   direction_id     544 non-null    int64  
 5   shape_id         544 non-null    object 
 6   exceptional      544 non-null    int64  
 7   route_long_name  544 non-null    object 
 8   route_type       544 non-null    int64  
 9   stop_name        544 non-null    object 
 10  stop_lat         544 non-null    float64
 11  stop_lon         544 non-null    float64
 12  parent_station   544 non-null    object 
 13  zone_id          544 non-null    object 
 14  stop_id_numeric  544 non-null    object 
 15  hour             544 non-null    int32  
dtypes: float64(2), int32(1), int64(4), object(9)
memory 

In [58]:
metro_data = metro_data.merge(metro_hourly, on=['route_id', 'hour'], how='left')

In [59]:
metro_data

,stop_sequence,route_id,service_id,trip_headsign,direction_id,shape_id,exceptional,route_long_name,route_type,stop_name,stop_lat,stop_lon,parent_station,zone_id,stop_id_numeric,hour,number_of_trips
0,0,M1,M1,Młociny,0,M1/TP-MLO,0,Kabaty — Młociny,1,Kabaty,52.131983,21.065174,3282m,1,32821,0,14.0
1,0,M1,M1,Młociny,0,M1/TP-MLO,0,Kabaty — Młociny,1,1,52.131621,21.065989,3282m,1,32821,0,14.0
2,1,M1,M1,Młociny,0,M1/TP-MLO,0,Kabaty — Młociny,1,Natolin,52.141008,21.056510,3281m,1,32811,2,4.0
3,1,M1,M1,Młociny,0,M1/TP-MLO,0,Kabaty — Młociny,1,1,52.140646,21.057267,3281m,1,32811,2,4.0
4,2,M1,M1,Młociny,0,M1/TP-MLO,0,Kabaty — Młociny,1,Imielin,52.149136,21.046366,3280m,1,32801,4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,10,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,2,52.231686,20.966844,5005m,1,50052,19,83.0
540,11,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,Młynów,52.237640,20.960475,5028m,1,50282,21,57.0
541,11,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,2,52.237671,20.961693,5028m,1,50282,21,57.0
542,12,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,Księcia Janusza,52.239096,20.944110,5030m,1,50302,23,2.0


In [60]:
metro_data['trip_id'] = metro_data['route_id']

In [61]:
metro_hourly["hour"] = metro_hourly["hour"].astype(int)

In [63]:
stops_total_info["hour"].unique()

array([ 4,  5,  6,  7,  8, 14, 15,  3,  9, 10, 11, 12, 13, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  0])

In [64]:
# Adjusting hours to 0-23 format: 
stops_total_info["hour"] = stops_total_info["hour"].apply(lambda x: x - 24 if x >= 24 else x)
print(stops_total_info["hour"].unique())

[ 4  5  6  7  8 14 15  3  9 10 11 12 13 16 17 18 19 20 21 22 23  0  1  2]


In [65]:
stops_total_info["hour"] = stops_total_info["hour"].astype(int)

In [66]:
print(stops_total_info.columns)
print(metro_data.columns)

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id_x',
       'stop_sequence', 'pickup_type', 'drop_off_type', 'hour', 'route_id',
       'service_id', 'trip_headsign', 'direction_id', 'shape_id',
       'exceptional', 'route_long_name', 'route_type', 'stop_id_y',
       'stop_name', 'stop_lat', 'stop_lon', 'parent_station', 'zone_id',
       'clean_stop_id', 'stop_id_numeric'],
      dtype='object')
Index(['stop_sequence', 'route_id', 'service_id', 'trip_headsign',
       'direction_id', 'shape_id', 'exceptional', 'route_long_name',
       'route_type', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station',
       'zone_id', 'stop_id_numeric', 'hour', 'number_of_trips', 'trip_id'],
      dtype='object')


In [67]:
print(stops_total_info.info())
print(metro_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5042371 entries, 0 to 5042370
Data columns (total 24 columns):
 #   Column           Dtype  
---  ------           -----  
 0   trip_id          object 
 1   arrival_time     object 
 2   departure_time   object 
 3   stop_id_x        object 
 4   stop_sequence    int64  
 5   pickup_type      float64
 6   drop_off_type    float64
 7   hour             int32  
 8   route_id         object 
 9   service_id       object 
 10  trip_headsign    object 
 11  direction_id     int64  
 12  shape_id         object 
 13  exceptional      int64  
 14  route_long_name  object 
 15  route_type       int64  
 16  stop_id_y        object 
 17  stop_name        object 
 18  stop_lat         float64
 19  stop_lon         float64
 20  parent_station   object 
 21  zone_id          object 
 22  clean_stop_id    object 
 23  stop_id_numeric  object 
dtypes: float64(4), int32(1), int64(4), object(15)
memory usage: 1.0+ GB
None
<class 'pandas.core.frame.Dat

In [68]:
stops_total_info = stops_total_info.drop(columns=[
    'arrival_time', 'departure_time', 'stop_id_x', 'pickup_type', 
    'drop_off_type', 'stop_id_y','clean_stop_id' 
])

In [69]:
missing_cols = set(stops_total_info.columns) - set(metro_data.columns)
for col in missing_cols:
    metro_data[col] = None  

In [70]:
print(stops_total_info.dtypes)
print(metro_data.dtypes)

trip_id             object
stop_sequence        int64
hour                 int32
route_id            object
service_id          object
trip_headsign       object
direction_id         int64
shape_id            object
exceptional          int64
route_long_name     object
route_type           int64
stop_name           object
stop_lat           float64
stop_lon           float64
parent_station      object
zone_id             object
stop_id_numeric     object
dtype: object
stop_sequence        int64
route_id            object
service_id          object
trip_headsign       object
direction_id         int64
shape_id            object
exceptional          int64
route_long_name     object
route_type           int64
stop_name           object
stop_lat           float64
stop_lon           float64
parent_station      object
zone_id             object
stop_id_numeric     object
hour                 int32
number_of_trips    float64
trip_id             object
dtype: object


In [72]:
duplicates = stops_total_info.duplicated(subset=['trip_id', 'stop_id_numeric', 'hour'], keep=False)
print(f"Trips Duplicates: {duplicates.sum()}")

Trips Duplicates: 12418


In [73]:
stops_total_info = stops_total_info.drop_duplicates()


In [74]:
stops_total_info['number_of_trips']=1

In [77]:
stops_total_info['number_of_trips']= stops_total_info['number_of_trips'].astype(int)

In [78]:
final_data = pd.concat([stops_total_info, metro_data], ignore_index=True)

In [83]:
metro_data['hour'].unique()

array([ 0,  2,  4,  6,  8, 10, 12, 13, 15, 17, 18, 21, 22, 24, 27, 30, 32,
       33, 35, 37, 39,  7,  9, 26, 29, 31,  5, 11, 19, 23, 14, 16])

In [79]:
final_data

,trip_id,stop_sequence,hour,route_id,service_id,trip_headsign,direction_id,shape_id,exceptional,route_long_name,route_type,stop_name,stop_lat,stop_lon,parent_station,zone_id,stop_id_numeric,number_of_trips
0,RA210929/1/TD-3ANNO/DP/04.13,0,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,Banacha — Annopol,0,Woronicza 04,52.189785,21.001984,NaN,1,324004,1.0
1,RA210929/1/TD-3ANNO/DP/04.13,1,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,Banacha — Annopol,0,Wołoska 04,52.195111,21.001824,NaN,1,323904,1.0
2,RA210929/1/TD-3ANNO/DP/04.13,2,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,Banacha — Annopol,0,Wołoska - Szpital 04,52.198657,21.001319,NaN,1,323804,1.0
3,RA210929/1/TD-3ANNO/DP/04.13,3,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,Banacha — Annopol,0,Kulskiego 04,52.202328,21.000267,NaN,1,323704,1.0
4,RA210929/1/TD-3ANNO/DP/04.13,4,4,1,RA210929/1/DP,Annopol,0,RA210929/1/TD-3ANNO,1,Banacha — Annopol,0,Rakowiecka - Sanktuarium 04,52.205617,20.999336,NaN,1,328604,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5042910,M2,10,19,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,2,52.231686,20.966844,5005m,1,50052,83.0
5042911,M2,11,21,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,Młynów,52.237640,20.960475,5028m,1,50282,57.0
5042912,M2,11,21,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,2,52.237671,20.961693,5028m,1,50282,57.0
5042913,M2,12,23,M2,M7,Księcia Janusza,1,M2/TP-KSJ,0,Księcia Janusza — Trocka,1,Księcia Janusza,52.239096,20.944110,5030m,1,50302,2.0


In [81]:
final_data['hour'].unique()

array([ 4,  5,  6,  7,  8, 14, 15,  3,  9, 10, 11, 12, 13, 16, 17, 18, 19,
       20, 21, 22, 23,  0,  1,  2, 24, 27, 30, 32, 33, 35, 37, 39, 26, 29,
       31])

In [80]:
final_data.to_csv('final_data.csv', sep=',', encoding='utf-8-sig', index=False)